In [21]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss


In [22]:
from lasagne.layers import DenseLayer
from lasagne.layers import InputLayer
from lasagne.layers import DropoutLayer
from lasagne.nonlinearities import softmax
from lasagne.updates import nesterov_momentum
from nolearn.lasagne import NeuralNet
from sklearn.cross_validation import StratifiedShuffleSplit


In [23]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

X = train_df.values.copy()
test_X = test_df.values.copy()
np.random.shuffle(X)
X, labels = X[:, 1:-1].astype(np.float32), X[:, -1]

encoder = LabelEncoder()
y = encoder.fit_transform(labels).astype(np.int32)

### we need a test set that we didn't train on to find the best weights for combining the classifiers
sss = StratifiedShuffleSplit(y, 1, test_size=0.1, random_state=1234)

for train_index, test_index in sss:
    print 'split the training data'


scaler = StandardScaler()
X = scaler.fit_transform(X)
train_X, train_y = X[train_index], y[train_index]
check_X, check_y = X[test_index], y[test_index]


test_X, test_ids = test_X[:, 1:].astype(np.float32), test_X[:, 0].astype(str)
test_X = scaler.transform(test_X)

split the training data


In [24]:
print 'training data: ', train_X.shape
print 

num_classes = len(encoder.classes_)
num_features = train_X.shape[1]

training data:  (55690, 93)



In [29]:

layers0 = [('input', InputLayer),
           ('dense0', DenseLayer),
           ('dropout', DropoutLayer),
           ('dense1', DenseLayer),
           ('output', DenseLayer)]


net0 = NeuralNet(layers=layers0,
                 
                 input_shape=(None, num_features),
                 dense0_num_units=200,
                 dropout_p=0.4,
                 dense1_num_units=200,
                 output_num_units=num_classes,
                 output_nonlinearity=softmax,
                 
                 update=nesterov_momentum,
                 update_learning_rate=0.01,
                 update_momentum=0.9,
                 
                 eval_size=0.2,
                 verbose=1,
                 max_epochs=50)


net0.fit(train_X, train_y)

  DenseLayer        	(None, 9)           	produces       9 outputs
  DenseLayer        	(None, 200)         	produces     200 outputs
  DropoutLayer      	(None, 200)         	produces     200 outputs
  DenseLayer        	(None, 200)         	produces     200 outputs
  InputLayer        	(None, 93)          	produces      93 outputs

 Epoch  |  Train loss  |  Valid loss  |  Train / Val  |  Valid acc  |  Dur
--------|--------------|--------------|---------------|-------------|-------
     1  |    0.908080  |    0.697891  |     1.301178  |     72.47%  |  1.0s
     2  |    0.686592  |    0.636604  |     1.078522  |     75.40%  |  1.0s
     3  |    0.649086  |    0.608596  |     1.066530  |     76.54%  |  1.0s
     4  |    0.623866  |    0.594349  |     1.049662  |     76.76%  |  1.0s
     5  |    0.608625  |    0.583194  |     1.043606  |     77.44%  |  1.0s
     6  |    0.593090  |    0.572651  |     1.035693  |     77.57%  |  1.1s
     7  |    0.583419  |    0.564730  |     1.033093  | 

NeuralNet(X_tensor_type=<function matrix at 0x10766d668>,
     batch_iterator_test=<nolearn.lasagne.BatchIterator object at 0x107e68b50>,
     batch_iterator_train=<nolearn.lasagne.BatchIterator object at 0x107e68b10>,
     dense0_num_units=200, dense1_num_units=200, dropout_p=0.4,
     eval_size=0.2, input_shape=(None, 93),
     layers=[('input', <class 'lasagne.layers.input.InputLayer'>), ('dense0', <class 'lasagne.layers.dense.DenseLayer'>), ('dropout', <class 'lasagne.layers.noise.DropoutLayer'>), ('dense1', <class 'lasagne.layers.dense.DenseLayer'>), ('output', <class 'lasagne.layers.dense.DenseLayer'>)],
     loss=<function negative_log_likelihood at 0x107d90de8>, max_epochs=50,
     more_params={}, on_epoch_finished=(), on_training_finished=(),
     output_nonlinearity=<function softmax at 0x107c83848>,
     output_num_units=9, regression=False,
     update=<function nesterov_momentum at 0x107d70140>,
     update_learning_rate=0.01, update_momentum=0.9,
     use_label_encoder=Fa

In [30]:
y_prob = net0.predict_proba(check_X)
print('LogLoss {score}'.format(score=log_loss(check_y, y_prob)))

LogLoss 0.510064097173


In [12]:
print type(y_prob)
y_prob.shape

<type 'numpy.ndarray'>


(144368, 9)